In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
!pip install pyro-ppl
plt.style.use("ggplot")
import pyro
import pyro.infer as infer
import pyro.distributions as dist
import pyro.nn as nn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/downloadDrive/csit-ex2-7/Day3')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X = np.load('A.npy')
Train_data = torch.tensor(X, dtype=torch.float32, device=device)
print(Train_data.shape)
torch.set_default_tensor_type('torch.cuda.FloatTensor')

torch.Size([305, 1217])


In [ ]:
from torch.distributions import constraints

In [ ]:
class NMF(pyro.nn.PyroModule):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super().__init__()
    self.N = output_dim
    self.D = input_dim
    self.M = hidden_dim
    self.aW = pyro.nn.PyroParam(torch.tensor(1.0), constraint=constraints.positive)
    self.bW = pyro.nn.PyroParam(torch.tensor(1.0), constraint=constraints.positive)
    self.aH = pyro.nn.PyroParam(torch.tensor(1.0), constraint=constraints.positive)
    self.bH = pyro.nn.PyroParam(torch.tensor(1.0), constraint=constraints.positive)
    self.W = pyro.nn.PyroSample(lambda self: dist.Gamma(self.aW, self.bW).expand([self.D, self.M]).to_event(2))
    self.H = pyro.nn.PyroSample(lambda self: dist.Gamma(self.aH, self.bH).expand([self.M, self.N]).to_event(2))
    self.d_axis = pyro.plate("d_axis", self.D, dim=-2)
    self.n_axis = pyro.plate("n_axis", self.N, dim=-1)
  
  def forward(self, X=None):
    S = pyro.sample("S", dist.Poisson(self.W @ self.H).to_event(2), obs=X)
    return S

In [ ]:
model = NMF(Train_data.shape[0], 10, Train_data.shape[1])
nuts = infer.NUTS(model)

mcmc = infer.MCMC(nuts, num_samples=2500, warmup_steps=200)
mcmc.run(Train_data)

Sample: 100%|██████████| 2700/2700 [13:31,  3.33it/s, step size=7.69e-02, acc. prob=0.840]


In [ ]:
print(model.W)
print(model.H)
print(model.W @ model.H)

tensor([[0.5757, 0.2117, 0.1790,  ..., 1.7090, 2.4301, 0.3592],
        [0.8965, 0.5606, 2.0569,  ..., 1.4096, 1.6390, 0.6573],
        [0.2494, 0.7196, 0.0796,  ..., 0.4872, 0.4511, 1.7808],
        ...,
        [0.9016, 0.0741, 1.6577,  ..., 0.0596, 0.1439, 1.8851],
        [0.2843, 0.5827, 0.7093,  ..., 0.4410, 1.0808, 0.6465],
        [0.7317, 0.4934, 1.4336,  ..., 3.4951, 0.8098, 0.0520]],
       grad_fn=<DivBackward0>)
tensor([[9.4059e-02, 9.5400e-01, 3.9383e+00,  ..., 1.3894e+00, 8.6642e-01,
         1.6322e+00],
        [1.9669e-01, 2.6711e-02, 7.5108e-01,  ..., 1.0288e+00, 1.6137e+00,
         3.0141e+00],
        [2.6476e+00, 5.8442e-01, 9.6470e-01,  ..., 6.1462e-02, 1.1458e+00,
         1.0916e+00],
        ...,
        [2.2663e-01, 4.0113e-01, 1.0389e+00,  ..., 2.8264e-01, 2.2427e+00,
         1.1233e+00],
        [2.9974e-03, 2.3702e+00, 4.7548e-01,  ..., 2.3528e+00, 8.6135e-01,
         2.7713e+00],
        [7.7152e-02, 6.8342e-01, 1.8576e+00,  ..., 1.1185e+00, 1.0617e-01

In [ ]:
W = model.W
H = model.H

In [ ]:
dell = (X - (W@H).detach().cpu().numpy())

In [ ]:
np.linalg.norm(dell)

6840.007289300583

In [ ]:
np.save("W_NMF_posson_K=10",W.detach().cpu().numpy())
np.save("H_NMF_posson_K=10", H.detach().cpu().numpy())

In [ ]:
print(W.shape, H.shape)

torch.Size([305, 10]) torch.Size([10, 1217])
